# OpenAI 외 도구 호출 에이전트(Tool Calling Agent)

OpenAI 외에도 `Anthropic`, `Google Gemini`, `Together.ai`, `Ollama`, `Mistral`과 같은 더 광범위한 공급자 구현을 지원합니다.

이번 챕터에서는 다양한 LLM 을 사용하여 도구 호출 에이전트를 생성하고 실행하는 방법을 살펴보겠습니다.

**참고 링크**

- [LangChain 공식 도큐먼트](https://python.langchain.com/v0.1/docs/modules/agents/agent_types/tool_calling/)

In [6]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

Python-dotenv could not parse statement starting at line 8


True

In [8]:
from langchain.tools import tool
from typing import List, Dict
from google import GoogleNews


# 도구 정의
@tool
def search_news(query: str) -> List[Dict[str, str]]:
    """Search Google News by input keyword"""
    news_tool = GoogleNews()
    return news_tool.search_by_keyword(query, k=5)


print(f"도구 이름: {search_news.name}")
print(f"도구 설명: {search_news.description}")

도구 이름: search_news
도구 설명: Search Google News by input keyword


In [9]:
# tools 정의
tools = [search_news]

## Agent 용 프롬프트 생성

- `chat_history` : 이전 대화 내용을 저장하는 변수 (멀티턴을 지원하지 않는다면, 생략 가능합니다.)
- `agent_scratchpad` : 에이전트가 임시로 저장하는 변수
- `input` : 사용자의 입력

In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import create_tool_calling_agent

# 프롬프트 생성
# 프롬프트는 에이전트에게 모델이 수행할 작업을 설명하는 텍스트를 제공합니다. (도구의 이름과 역할을 입력)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `search_news` tool for searching keyword related news.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

## Tool Calling 을 지원하는 다양한 LLM 목록

실습 진행을 위해서는 아래 내용을 설정해야 합니다.

**Anthropic**

- [Anthropic API 키 발급 관련](https://console.anthropic.com/settings/keys)
- `.env` 파일 내 `ANTHROPIC_API_KEY` 에 발급받은 키를 설정하세요

**Gemini**

- [Gemini API 키 발급 관련](https://aistudio.google.com/app/apikey?hl=ko)
- `.env` 파일 내 `GOOGLE_API_KEY` 에 발급받은 키를 설정하세요

**Together AI**

- [Together AI API 키 발급 관련](https://api.together.ai/)
- `.env` 파일 내 `TOGETHER_API_KEY` 에 발급받은 키를 설정하세요

**Ollama**

- [Ollama Tool Calling 지원 모델 리스트](https://ollama.com/search?c=tools)
- [이번 실습에 사용할 llama3.1 모델](https://ollama.com/library/llama3.1)
- 터미널 창에 `ollama pull llama3.1` 명령어를 입력하여 모델을 다운로드 받습니다.
- 이전에 Ollama 를 사용하지 않았다면, [Ollama](https://wikidocs.net/233805) 를 참고해 주세요.

langchain-ollama 설치를 한 뒤 진행해 주세요.

In [11]:
# !pip install -qU langchain-ollama==0.1.3

In [14]:
# from langchain_anthropic import ChatAnthropic
# from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
import os

# GPT-4o-mini
gpt = ChatOpenAI(model="gpt-4o-mini")

# # Claude-3-5-sonnet
# claude = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=0)

# # Gemini-1.5-pro-latest
# gemini = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0)

# # Llama-3.1-70B-Instruct-Turbo
# llama = ChatOpenAI(
#     base_url="https://api.together.xyz/v1",
#     api_key=os.environ["TOGETHER_API_KEY"],
#     model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
# )

# Llama-3.1
ollama = ChatOllama(model="llama3.1", temperature=0)

# Qwen2.5 7B (한글 성능 괜찮은 편)
qwen = ChatOllama(
    model="qwen2.5:latest",
)

LLM 기반으로 Agent 를 생성합니다.

In [15]:
from langchain.agents import create_tool_calling_agent

# Agent 생성
gpt_agent = create_tool_calling_agent(gpt, tools, prompt)
# claude_agent = create_tool_calling_agent(claude, tools, prompt)
# gemini_agent = create_tool_calling_agent(gemini, tools, prompt)
# llama_agent = create_tool_calling_agent(llama, tools, prompt)
ollama_agent = create_tool_calling_agent(ollama, tools, prompt)
qwen_agent = create_tool_calling_agent(qwen, tools, prompt)

## AgentExecutor 생성 후 실행 및 결과 확인



In [16]:
from langchain.agents import AgentExecutor

# gpt_agent 실행
agent_executor = AgentExecutor(
    agent=gpt_agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
)

result = agent_executor.invoke({"input": "AI 투자와 관련된 뉴스를 검색해 주세요."})

print("Agent 실행 결과:")
print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `search_news` with `{'query': 'AI 투자'}`


[{'url': 'https://news.google.com/rss/articles/CBMiX0FVX3lxTE5BM01hbENVMXVydWowNlFXNld1alN6bjNjaFZLOG9mcms5cnRlbjlPUTNyWEFKdFNPMGRWS3FUclBRelJ5V3ZVbzVZbF9xZVV6bzJaZGZ6V2JhanU5emlF?oc=5', 'content': "퓨리오사AI 투자 네이버…메타와 AI칩 매개 '공동전선' 길 열리나 - 뉴스1"}, {'url': 'https://news.google.com/rss/articles/CBMiYEFVX3lxTE9jdE0tTVNBNTZfY0VOUURoODl2clczV1BHRGNkSzl2aWVwUHg3c21GVTUybGVlMTZHT1g3Qm9EZTJYM3ctQXluMThmQzh4aDBiQlZhUDRhb0JTemJSV0N4Yg?oc=5', 'content': '‘100배 잭팟 터질까’…퓨리오사AI 투자 VC 기대감 - 아시아경제'}, {'url': 'https://news.google.com/rss/articles/CBMigAFBVV95cUxNSXMxb0ZmUEZWSHVHY1FvNjVjN2owSmd2aDRJNy0tbC1MZDhGVG5kTktnOFZBRVVPeFBuZUFkUnpITFJRdHFGeURvQkhGTXJaSnEwU3ZhV2FWNWpWSmpfWTVEVEFUazdlRW5kell1TXZGT2w3bjJjZXRJeENwWDNsVg?oc=5', 'content': '"메타, 韓 AI 스타트업 퓨리오사AI 이달 내 인수 완료할듯" - 이데일리'}, {'url': 'https://news.google.com/rss/articles/CBMicEFVX3lxTE1URmtseUtxcExleEcxYXpJRUZ6N05jbkJvRUhlWkVaVmxXNVlfbXhKWVFwVkg4dmVZX2V

다양한 llm을 사용하여 에이전트를 실행합니다.

다음은 입력받은 llm을 사용하여 Agent 를 생성하고 실행하여 결과를 출력하는 함수입니다.

In [17]:
def execute_agent(llm, tools, input_text, label):
    agent = create_tool_calling_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools, verbose=False)
    result = executor.invoke({"input": input_text})
    print(f"[{label}] 결과입니다.")
    if isinstance(result["output"], list) and len(result["output"]) > 0:
        for item in result["output"]:
            if "text" in item:
                print(item["text"])
    elif isinstance(result["output"], str):
        print(result["output"])
    else:
        print(result["output"])

각 llm 별로 에이전트를 생성하고 실행하여 결과를 출력합니다.

In [18]:
query = (
    "AI 투자와 관련된 뉴스를 검색하고, 결과를 Instagram 게시글 형식으로 작성해 주세요."
)

In [19]:
# gpt
execute_agent(gpt, tools, query, "gpt")

[gpt] 결과입니다.
---

📰 **AI 투자 현황** 🔍

1️⃣ **퓨리오사AI, 네이버와의 투자**  
메타와 AI 칩을 통한 공동전선이 열렸습니다. 💡 [자세히 보기](https://news.google.com/rss/articles/CBMiX0FVX3lxTE5BM01hbENVMXVydWowNlFXNld1alN6bjNjaFZLOG9mcms5cnRlbjlPUTNyWEFKdFNPMGRWS3FUclBRelJ5V3ZVbzVZbF9xZVV6bzJaZGZ6V2JhanU5emlF?oc=5)

2️⃣ **VC 기대감 상승**  
'100배 잭팟 터질까?' 퓨리오사AI에 대한 기대가 점점 커지고 있습니다! 🚀 [자세히 보기](https://news.google.com/rss/articles/CBMiYEFVX3lxTE9jdE0tTVNBNTZfY0VOUURoODl2clczV1BHRGNkSzl2aWVwUHg3c21GVTUybGVlMTZHT1g3Qm9EZTJYM3ctQXluMThmQzh4aDBiQlZhUDRhb0JTemJSV0N4Yg?oc=5)

3️⃣ **메타의 스타트업 인수**  
메타가 한국 AI 스타트업 퓨리오사AI를 이달 내 인수할 계획입니다. 🔑 [자세히 보기](https://news.google.com/rss/articles/CBMigAFBVV95cUxNSXMxb0ZmUEZWSHVHY1FvNjVjN2owSmd2aDRJNy0tbC1MZDhGVG5kTktnOFZBRVVPeFBuZUFkUnpITFJRdHFGeURvQkhGTXJaSnEwU3ZhV2FWNWpWSmpfWTVEVEFUazdlRW5kell1TXZGT2w3bjJjZXRJeENwWDNsVg?oc=5)

4️⃣ **빅테크 기업들의 인프라 부족**  
AI 투자가 확대되더라도 인프라 부족에 대한 우려가 존재합니다. ⚠️ [자세히 보기](https://news.google.com/rss/articles/CBMicEFVX3lxTE1URmtseUtxcExleEcxYXpJRUZ6N05jbkJvRUhlWkVaVmxXNVlf

In [20]:
# claude
# execute_agent(claude, tools, query, "claude")

In [21]:
# gemini
# execute_agent(gemini, tools, query, "gemini")

In [22]:
# # llama3.1 70B (Together.ai)
# execute_agent(
#     llama,
#     tools,
#     "Search AI related news and write it in Instagram post format",
#     "llama3.1 70B",
# )

In [24]:
# llama3.1 8B (ollama)
execute_agent(ollama, tools, query, "llama3.1(Ollama)")

[llama3.1(Ollama)] 결과입니다.
**AI 투자 뉴스**

퓨리오사, 메타와 AI칩 매개 '공동전선' 길 열리나?

* 퓨리오사는 메타와 협력하여 AI칩을 개발 중입니다.
* 이 칩은 AI를 위한 고성능 프로세서로, 향후 AI 투자에 큰 영향을 미칠 것으로 예상됩니다.

‘100배 잭팟 터질까’…퓨리오사AI 투자 VC 기대감

* 퓨리오사의 AI 투자가 성공할 경우, 100배 이상의 수익을 거둘 수 있습니다.
* 이에 따라 VC들은 퓨리오사를 주목하고 있습니다.

‘메타, 韓 AI 스타트업 퓨리오사AI 이달 내 인수 완료할듯’

* 메타는 퓨리오사의 AI를 인수할 예정입니다.
* 이로써 메타는 한국의 AI 기술을 확보할 수 있을 것입니다.

빅테크 기업들, AI 투자 확대에도 인프라 부족 우려

* 빅테크 기업들은 AI 투자를 확대하고 있지만, 인프라 부족으로 인해 어려움을 겪고 있습니다.
* 이에 따라 기업들은 새로운 기술과 솔루션을 개발 중입니다.

빅테크 AI투자 과잉우려속 투자 안한 애플 반사이익

* 애플은 빅테크 기업들처럼 AI 투자를 확대하지 않았습니다.
* 그러나 이로써 애플은 다른 기업들과 비교하여 더 큰 수익을 거둘 수 있습니다.


In [26]:
# qwen2.5 7B (ollama)
query = "AI 투자와 관련된 뉴스를 검색하고, 결과를 Instagram 게시글 형식으로 작성해 주세요. 한글로 답변하세요!"

execute_agent(qwen, tools, query, "qwen2.5(Ollama)")

[qwen2.5(Ollama)] 결과입니다.
AI 투자를 주제로 한 최신 뉴스를 찾아보았습니다.

---

 PUB: 

### 퓨리오사 AI 투자, 메타와 AI 칩 매개 '공동전선' 길 열리나

- **출처:** 뉴스1
- **링크:** [https://news.google.com/rss/articles/CBMiX0FVX3lxTE5BM01hbENVMXVydWowNlFXNld1alN6bjNjaFZLOG9mcms5cnRlbjlPUTNyWEFKdFNPMGRWS3FUclBRelJ5V3ZVbzVZbF9xZVV6bzJaZGZ6V2JhanU5emlF?oc=5](https://news.google.com/rss/articles/CBMiX0FVX3lxTE5BM01hbENVMXVydWowNlFXNld1alN6bjNjaFZLOG9mcms5cnRlbjlPUTNyWEFKdFNPMGRWS3FUclBRelJ5V3ZVbzVZbF9xZVV6bzJaZGZ6V2JhanU5emlF?oc=5)

---

### '100배 잭팟 터질까'…퓨리오사AI 투자 VC 기대감

- **출처:** 아시아경제
- **링크:** [https://news.google.com/rss/articles/CBMiYEFVX3lxTE9jdE0tTVNBNTZfY0VOUURoODl2clczV1BHRGNkSzl2aWVwUHg3c21GVTUybGVlMTZHT1g3Qm9EZTJYM3ctQXluMThmQzh4aDBiQlZhUDRhb0JTemJSV0N4Yg?oc=5](https://news.google.com/rss/articles/CBMiYEFVX3lxTE9jdE0tTVNBNTZfY0VOUURoODl2clczV1BHRGNkSzl2aWVwUHg3c21GVTUybGVlMTZHT1g3Qm9EZTJYM3ctQXluMThmQzh4aDBiQlZhUDRhb0JTemJSV0N4Yg?oc=5)

---

### 자체 칩 개발 실패한 메타... 퓨리오사AI M&A 성사될까?

- **출처:** 조선비즈 - 조선비즈
- **링크:** [https://n